In [159]:
import tensorflow as tf
import cv2
import os
import numpy as np

In [152]:
def getList(file_path):
    
    print('getList is ok')
    
    DataToPros = os.listdir(file_path+'/')
    
    return DataToPros

def getFile(file_path,file_list):
    
    print('getFile is ok')
    
    data = []
    label = []
    for i in file_list:
        data.append(file_path+'/'+i)
        if file_path == 'pics_modal/rumor_pics':
            label.append(0)
        else:
            label.append(1)
            
    return data,label

def proList():
    
    print('proList is ok')

    file_path_rumor = "pics_modal/rumor_pics"
    file_path_truth = "pics_modal/truth_pics"
    list_rumor = getList(file_path_rumor)
    list_truth = getList(file_path_truth)
    
    rd,rl = getFile(file_path_rumor,list_rumor[:20])
    td,tl = getFile(file_path_truth,list_truth[:20])
    
    prodata = rd+td
    prolabel = rl+tl
    
    temp = np.array([prodata,prolabel])
    temp = temp.T
    np.random.shuffle(temp)
    
    prodata = list((temp[:,0]))
    prolabel = list((temp[:,1]))
    prolabel = [int(float(i)) for i in prolabel]

    return prodata, prolabel
    

def getBat(data_list,label_list,width,height,batSize,capacity):
    
    print('getBat is ok')

    image = []
    
    for i in data_list:
        
        temp = tf.read_file(str(i))
        temp = tf.image.decode_jpeg(temp,channels = 3)
        temp = tf.image.resize_image_with_crop_or_pad(temp,
                                                       width,height)
    
        temp = tf.image.per_image_standardization(temp)
        image.append(temp)
        
    label = tf.cast(label_list,tf.int32)
    image = tf.cast(image,tf.float32)
    return image,label
#     image_batch,label_batch = tf.train.batch([image,label_list],
#             batch_size = batSize,
#             num_threads = 1,
#             capacity = capacity)
#     images_batch = tf.cast(image_batch,tf.float32)
#     labels_batch = tf.reshape(label_batch,[batSize])
    
#     return images_batch,label_batch

In [153]:
def init_weights(shape):
    res = tf.Variable(tf.random_normal(shape,stddev = 0.01))
    return res

def norm(x,lsize = 4):
    return tf.nn.lrn(x,depth_radius = lsize,bias = 1,
                      alpha = 0.001/9.0,beta = 0.75)

def conv2d(x, w, b):
    x = tf.nn.conv2d(x,w,strides = [1,1,1,1],padding = "SAME")
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)    
    # tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
    # x(input)  : [batch, in_height, in_width, in_channels]
    # W(filter) : [filter_height, filter_width, in_channels, out_channels]
    # strides   : The stride of the sliding window for each dimension of input.
    #             For the most common case of the same horizontal and vertices strides, 
    #             strides = [1, stride, stride, 1]

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1],
                          strides = [1, 2, 2, 1], padding = 'SAME')
    # tf.nn.max_pool(value, ksize, strides, padding, data_format='NHWC', name=None)
    # x(value)              : [batch, height, width, channels]
    # ksize(pool大小)        : A list of ints that has length >= 4. 
    #     The size of the window for each dimension of the input tensor.
    # strides(pool滑动大小)   : A list of ints that has length >= 4. 
    #   The stride of the sliding window for each dimension of the input tensor.

def loss(logits,label_batches):
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                                    labels=label_batches)
    loss = tf.reduce_mean(cross_entropy)
    
    return loss
 
def get_accuracy(logits,labels):
    
    acc = tf.nn.in_top_k(logits,labels,1)
    acc = tf.cast(acc,tf.float32)
    acc = tf.reduce_mean(acc)
    
    return acc

In [154]:
def mmodel(data):

    # init w      
    weights = {
      "w1":init_weights([3,3,3,16]),
      "w2":init_weights([3,3,16,128]),
      "w3":init_weights([3,3,128,256]),
      "w4":init_weights([4096,4096]),
      "wo":init_weights([4096,2])}
     
    # init biases
    biases = {
     "b1":init_weights([16]),
     "b2":init_weights([128]),
     "b3":init_weights([256]),
     "b4":init_weights([4096]),
     "bo":init_weights([2])
     }


    layer1 = conv2d(data,weights["w1"],biases["b1"])
    layer2 = max_pool_2x2(layer1)
    layer2 = tf.nn.lrn(layer2)
    layer3 = conv2d(layer2,weights["w2"],biases["b2"])
    layer4 = max_pool_2x2(layer3)
    layer4 = tf.nn.lrn(layer4)
    layer5 = conv2d(layer4,weights["w3"],biases["b3"])
    layer6 = max_pool_2x2(layer5)
    layer6 = tf.reshape(layer6,[-1,weights["w4"].get_shape().as_list()[0]])
    layer7 = tf.nn.relu(tf.matmul(layer6,weights["w4"])+biases["b4"])
    softmax = tf.add(tf.matmul(layer7,weights["wo"]),biases["bo"])
    
    return softmax

In [157]:
def training(loss,lr):
    train_op = tf.train.RMSPropOptimizer(lr,0.9).minimize(loss)
    return train_op

def cnn_run(data_bat,label_bat):
    log_dir = '/Users/yanghang/Desktop/Dataset/picSort/'
    p = mmodel(data_bat)
    cost = loss(p,label_bat)
    train_op = training(cost,0.01)
    acc = get_accuracy(p,label_bat)
    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    saver = tf.train.Saver()
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(sess = sess,coord = coord)

    try:
        for step in np.arange(20):
            print(step)
#             if coord.should_stop():
#                 break
            _,train_acc,train_loss = sess.run([train_op,acc,cost])
            print("loss:{} accuracy:{}".format(train_loss,train_acc))
            if step % 100 == 0:
                check = os.path.join(log_dir,"model.ckpt")
                saver.save(sess,check,global_step = step)
                
    except tf.errors.OutOfRangeError:
        print('Wrong!!!')
        
#     finally:
#         coord.request_stop()
#     coord.join(threads)
    sess.close()
        
    
def main():
    
    data, label = proList()
    data_bat, label_bat = getBat(data,label,32,32,5,64)
    cnn_run(data_bat,label_bat)

if __name__ == 'main':
    main()

In [158]:
main()

proList is ok
getList is ok
getList is ok
getFile is ok
getFile is ok
getBat is ok
0
loss:0.6933293342590332 accuracy:0.5
1
loss:0.6933252215385437 accuracy:0.5
2
loss:0.6933209300041199 accuracy:0.5
3
loss:0.6933164596557617 accuracy:0.5
4
loss:0.6933118104934692 accuracy:0.5
5
loss:0.6933070421218872 accuracy:0.5
6
loss:0.6933020353317261 accuracy:0.5
7
loss:0.6932969093322754 accuracy:0.5
8
loss:0.6932915449142456 accuracy:0.5
9
loss:0.6932859420776367 accuracy:0.5
10
loss:0.6932801008224487 accuracy:0.5
11
loss:0.6932740211486816 accuracy:0.5
12
loss:0.6932677030563354 accuracy:0.5
13
loss:0.6932612061500549 accuracy:0.5
14
loss:0.6932546496391296 accuracy:0.5
15
loss:0.6932479739189148 accuracy:0.5
16
loss:0.6932411193847656 accuracy:0.5
17
loss:0.6932340264320374 accuracy:0.5
18
loss:0.6932266354560852 accuracy:0.5
19
loss:0.6932187080383301 accuracy:0.5


In [ ]:
print('1')